# Imports

In [7]:
#imports
import os
import torch
from pyannote.audio import Pipeline
from pydub import AudioSegment
import matplotlib.pyplot as plt
# from audioProcessing import trim_and_save_wav
from timeConversion import time_to_miliseconds, seconds_to_miliseconds
from videoAudioData import video_data
from audioProcessing import read_audio_meta_from_json, reorder_labels, calculate_accuracy, get_intervals_from_audio,combine_random_audio_clips, add_noise_to_combined_files
from constants import API_KEY_SPEAKER_DIARIZATION_READ
import json

access_token = API_KEY_SPEAKER_DIARIZATION_READ

# DATA
The data source files are excluded but store it in data/videoAudio/[name]

0. video_audio_0.wav - https://www.youtube.com/watch?v=Xnk4seEHmgw - Jim's Pranks Against Dwight - The Office US
1. video_audio_1.wav - https://www.youtube.com/watch?v=-uleG_Vecis - 100 Computer Science Concepts Explained
2. video_audio_2.wav - https://www.youtube.com/watch?v=hxi4RzZaO04 - How She Made $88 Million From A Pilates Company
3. video_audio_3.wav - https://www.youtube.com/watch?v=zaB_20bkoA4 - Elon Musk's BRUTALLY Honest Interview With Tucker Carlson (2023)
4. video_audio_4.wav - https://www.youtube.com/watch?v=o3K_HbpWNpg - Think Fast. Talk Smart | Matt Abrahams | TEDxMontaVistaHighSchool
5. video_audio_5.wav - https://www.youtube.com/watch?v=ssgRCoRh_EY - Dua Lipa on Her Very Detailed Daily Schedule, Notebook of Song Lyrics, New Album & Albanian Sayings
6. video_audio_6.wav - https://www.youtube.com/watch?v=fZLLqapzxRQ - Little Big Shots Meet Micro Mayor James Episode Highlight( Engsub)
7. video_audio_7.wav - https://www.youtube.com/watch?v=WttV8_J9674 - Kardashian-Jenner Sisters' BIGGEST Fights | KUWTK | E!
8. video_audio_8.wav - https://www.youtube.com/watch?v=uOOCL7PLNNo - Will Gordon Ramsay Find the Lamb Sauce Cooking Lamb Chops?
9. video_audio_9.wav - https://www.youtube.com/watch?v=noR_3lkeAoY - Show-Stopping Lamb Dish Stuns Marcus Wareing | MasterChef UK




## Splitting into the audio tracks

Type:
woman - denotes woman speech;
man - denotes man speech;
kid - denotes kid speech;

Tembre:
m - denotes male speech;
f - denotes female speech;

Duration:
s - short for not longer than 5s
l - for longer than 5s

example: woman1_f_s_1.wav - denotes woman 1 is speaking, female, for shorter than 6s, 1st audio track

In [2]:
import numpy as np


for audio_clip in video_data:
  t1 = time_to_miliseconds(audio_clip[5]) #Works in milliseconds
  t2 = time_to_miliseconds(audio_clip[6])
 
  new_audio = AudioSegment.from_file(f'data/videoAudio/{audio_clip[0]}')
  new_audio = new_audio[t1:t2]
  # Convert audio to raw PCM data
  new_audio_raw = new_audio.raw_data

  # Convert raw PCM data to numpy array
  audio_array = np.frombuffer(new_audio_raw, dtype=np.int16)

  # Clip new_audio audio data to ensure it stays within the appropriate range [-32768, 32767]
  audio_array = np.clip(audio_array, -32768, 32767)

  # Convert the audio_array audio array back to bytes
  new_audio_raw = audio_array.astype(np.int16).tobytes()

  # Create a new AudioSegment from the noisy audio raw data
  if new_audio.channels == 1:
      # If the original audio is mono, set the new audio to mono
      n_audio = AudioSegment(new_audio_raw, frame_rate=new_audio.frame_rate, sample_width=new_audio.sample_width, channels=1)
  else:
      # If the original audio is stereo, set the new audio to stereo
      n_audio = AudioSegment(new_audio_raw, frame_rate=new_audio.frame_rate, sample_width=new_audio.sample_width, channels=2)
  
  # Save the new audio to a new .wav file
  n_audio.export(f'data/audioClips/{audio_clip[4]}.wav', format="wav") #Exports to a wav file in the current path

## Combine Clips

In [3]:
# Number of clips
number_clips = 500
combine_random_audio_clips(number_clips)

# Predict

Did extract the data from the videos. Concatenated randomly 30 combined audios. Normalized concatenated audios. Analyzed using tools.
Added noise. Analyzed the results. Gender and so on.

After inspecting the predicted labels I have observed speaker labels starting not from zero, sometimes the algorithm was predicting labels mixed or top to bottom, therefore I have wrote an algorithm to rename speakers based on the timing they were predicted. Which bumped the score significantly. 

## Pipeline

In [4]:
# Set the CUDA visible devices
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()

    if num_gpus <= 1:
        print(f'Using {num_gpus} GPU')
    else:
        print(f'Using {num_gpus} GPUs')

# You can also explicitly set a device
# For example, to use GPU 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=access_token)

pipeline.to(device)

torchvision is not available - cannot save figures


## Predict without noise

In [5]:
# for each created combined audio
for clip in range(number_clips):
  file_path = f'data/combinedAudioClips/audio_meta_{clip}.json'
  audio_meta = read_audio_meta_from_json(file_path)
  true_labels = get_intervals_from_audio(audio_meta)

  # apply pretrained pipeline
  diarization = pipeline(f'data/combinedAudioClips/combined_audio_{clip}.wav', num_speakers=len(true_labels))
  predicted_labels = []
  # predict labels
  for turn, _, speaker in diarization.itertracks(yield_label=True):
      prediction = f"start={seconds_to_miliseconds(turn.start)}ms stop={seconds_to_miliseconds(turn.end)}ms {speaker}"
      predicted_labels.append(prediction)

  # correct the order
  predicted_labels = reorder_labels(predicted_labels)

  accuracy = calculate_accuracy(true_labels, predicted_labels)

  predictions = {
     'true_labels': true_labels,
     'predicted_labels': predicted_labels,
     'accuracy': float(f'{accuracy:.3f}'),
     'number_of_speakers': len(true_labels)
  }
  # write results to json
  with open(f"data/predictions/preds_{clip}.json", "w") as file:
      json.dump(predictions, file)
  

Found only 5 clusters. Using a smaller value than 11 for `min_cluster_size` might help.


FileNotFoundError: [Errno 2] No such file or directory: 'data/combinedAudioClips/audio_meta_1.json'

## Predict with noise

In [ ]:
noise_levels = [0.01,0.05,0.1,0.3,0.5,0.7,0.9]

In [ ]:
for noise in noise_levels:
  add_noise_to_combined_files(noise,number_clips)  # Add % of noise to the combined audio
  # for each created combined audio
  for clip in range(number_clips):
    file_path = f'data/combinedAudioClips/audio_meta_{clip}.json'
    audio_meta = read_audio_meta_from_json(file_path)
    true_labels = get_intervals_from_audio(audio_meta)

    # apply pretrained pipeline
    diarization = pipeline(f'data/combinedAudioClips/combined_audio_noise_{str(noise)}_{clip}.wav', num_speakers=len(true_labels))
    predicted_labels = []
    # predict labels
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        prediction = f"start={seconds_to_miliseconds(turn.start)}ms stop={seconds_to_miliseconds(turn.end)}ms {speaker}"
        predicted_labels.append(prediction)

    # correct the order
    predicted_labels = reorder_labels(predicted_labels)

    accuracy = calculate_accuracy(true_labels, predicted_labels)

    predictions = {
      'true_labels': true_labels,
      'predicted_labels': predicted_labels,
      'accuracy': float(f'{accuracy:.3f}'),
      'number_of_speakers': len(true_labels)
    }
    # write results to json
    with open(f"data/predictions/preds_noise_{str(noise)}_{clip}.json", "w") as file:
        json.dump(predictions, file)

Found only 6 clusters. Using a smaller value than 12 for `min_cluster_size` might help.
Found only 3 clusters. Using a smaller value than 12 for `min_cluster_size` might help.
Found only 6 clusters. Using a smaller value than 12 for `min_cluster_size` might help.
